In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
import datetime
import os
current_time = datetime.datetime.now().date()
if current_time< datetime.date(2024, 1, 1):
    llm_model="The current date is before January 1, 2024. Exiting."
else:
    llm_model="model is ollma mistral"
    
print(llm_model)
    


model is ollma mistral


In [3]:
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
llm = OllamaEmbeddings(model="mistral")
loader =PyPDFLoader("research.pdf")
documents = loader.load()

C:\Users\Acer\AppData\Local\Temp\ipykernel_3480\3024412389.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  llm = OllamaEmbeddings(model="mistral")


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(documents)

In [5]:
text_splitter


In [ ]:
vectordb=Chroma.from_documents(
    
    embedding=llm,
    persist_directory="chroma_db_for_chat",
    collection_name="research_collection"
)
vectordb

In [8]:
vectordb.add_documents(split_documents)

['ecf92835-d4bf-400e-ab7d-8716c82c7722',
 'd5084c64-e4cb-4c63-ac5a-ce39f56f1652',
 'edfe7a0a-6660-4d6c-a46f-71563b7e936a',
 'da731837-53bd-4c3c-b554-d8ea623f1a76',
 'e0d27176-0661-452b-843c-af276b30e82e',
 '9f5f9287-9719-4c42-a243-bd085049770a',
 'd4485d49-b55c-4cbb-a945-436459211bde',
 'd4d8a6e0-dd74-4c85-b65e-77b7d2177810',
 '4c7aa973-525d-49ae-b2b3-5724346475ab',
 'd9e0ea25-0404-4278-8f0e-a7d58689001d',
 'd9b35fa1-3fe9-4129-9c59-a4c6243ce1ab',
 '9cd523fb-b495-46ed-b262-b06347b0b471',
 'a7c5914a-50d5-4381-800d-1177738dc332',
 '92167efa-4df9-4070-b70b-cc9e3255e9f8',
 'bb75def1-d776-44f0-abf9-2c8c797339fc',
 'dd1fc95f-80a3-45d1-a5d9-003277878468',
 '6b3efb55-056d-4307-a2ad-3b99450cd72a',
 '41c089b3-18da-4f4f-a039-d5348640ec94',
 '50b9d882-a16b-419c-ad41-7c7d21ea3800',
 '3fa46db7-2d47-451a-b1dc-7c48f32b43c1',
 '8df7021a-5cfd-47e2-8b1f-2bc644e5da9d',
 '21a4a993-81de-45aa-a4f5-3d122eb0177c',
 '3b38d163-f240-46ff-8f39-9a4c6a40a941',
 '6b964658-1731-4023-9957-abef1d2e2b00',
 '380c8175-aa5a-

In [9]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [11]:
from langchain_community.chat_models import ChatOllama

model = ChatOllama(model="mistral", temperature=0)
model.predict("Hello world!")

C:\Users\Acer\AppData\Local\Temp\ipykernel_3480\3619756395.py:4: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  model.predict("Hello world!")


" Hello there! It's nice to meet you. How can I assist you today?\n\nHere in this chat, we can discuss a wide range of topics such as programming, mathematics, science, history, literature, and more. If you have any specific questions or if you need help with something, feel free to ask! I'm here to help. 😊"

In [12]:
# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain
from langchain.chains import RetrievalQA
question = "Is probability a class topic?"
qa_chain = RetrievalQA.from_chain_type(model,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

C:\Users\Acer\AppData\Local\Temp\ipykernel_3480\205835925.py:18: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


' Yes, probability is a common topic in many classes, particularly in mathematics and statistics. It involves the study of random events and their likelihoods.'

In [13]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history",
    return_messages=True)

C:\Users\Acer\AppData\Local\Temp\ipykernel_3480\3824428362.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history",


In [14]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [15]:
question = "Is probability a class topic?"
result = qa({"question": question})

In [16]:
result['answer']

' Based on the provided context, it appears that the data presented is related to machine learning, specifically deep learning, and sign language recognition. However, the context does not explicitly mention "probability" as a class topic. It\'s possible that probability concepts might be used in the background of these studies (for example, in understanding the performance of the models), but it\'s not clear from the provided information whether probability is a specific focus or topic of study in this context.'

In [17]:
question = "why are those prerequesites needed?"
result = qa({"question": question})

In [19]:
result['answer']

' To study Machine Learning, particularly Deep Learning, and Sign Language Recognition, you would typically need the following prerequisites:\n\n1. Strong foundation in Mathematics, including Linear Algebra, Calculus, Probability, and Statistics.\n2. Programming skills, preferably in a language commonly used for Machine Learning such as Python or R. Familiarity with libraries like NumPy, Pandas, TensorFlow, PyTorch, or Keras would be beneficial.\n3. Knowledge of basic Machine Learning algorithms and concepts, such as Supervised Learning, Unsupervised Learning, Reinforcement Learning, Regression, Classification, Clustering, etc.\n4. Understanding of Deep Learning concepts, including Neural Networks, Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), Long Short-Term Memory (LSTM), and other deep learning architectures.\n5. Familiarity with Data Preprocessing techniques, such as Normalization, Standardization, Feature Extraction, etc.\n6. Knowledge of specific domains l